# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [6]:
!cat publications.tsv

pub_date	title	venue	 excerpt	citation	url_slug	paper_url
2023-01-01	Bayesian Clustering of n-gons via a Double Dirichlet Mixture Model	Journal of the American Statistical Association	The explosion of deep learning in medical image analysis has enabled the efficient abstraction of detailed biological shapes from medical images, creating an abundance of often-unlabeled biological shape datasets. Statistical characterization of these shapes is vital to medical image analysis and has informed clinical diagnosis; thus, as medical images grow increasingly larger and more detailed, there is a need for transparent statistical approaches to biological shape analysis that complement deep learning methods. Clustering is an unsupervised technique that partitions a dataset into similar subsets and has myriad medical applications. Attempting to cluster shapes presents unique challenges, including the need for shape representations that retain maximal geometric information. This study aims to develo

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-01-01,Bayesian Clustering of n-gons via a Double Dir...,Journal of the American Statistical Association,NaN,"Jin et al. (2023). ""Bayesian Clustering of n-g...",bacon,NaN
1,2023-01-01,Deep learning-based measurement of murine bone...,Scientific Reports,NaN,"Rong et al. (2023). ""Deep learning-based measu...",bone-length,NaN
2,2022-12-08,A Deep Learning Approach for Histology-Based N...,Modern Pathology,NaN,"Rong et al. (2023). ""A Deep Learning Approach ...",hd-yolo,NaN
3,2023-01-01,A deep learning-based Onion Peeling algorithm ...,Cancers,NaN,"Zhang et al. (2023). ""A deep learning-based On...",onion-peeling,NaN
4,2023-02-15,Deep Learning-Based Hepatic Ploidy Quantificat...,Genes,NaN,"Wen et al. (2023). ""Deep Learning-Based Hepati...",hepatic-ploidy,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [16]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    #if len(str(item.excerpt)) > 5:
    #    md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    #if len(str(item.excerpt)) > 5:
    #    md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [17]:
!ls ../_publications/

2022-12-08-hd-yolo.md        2023-01-01-onion-peeling.md
2023-01-01-bacon.md          2023-02-15-hepatic-ploidy.md
2023-01-01-bone-length.md


In [18]:
!cat ../_publications/2022-12-08-hd-yolo.md

---
title: "A Deep Learning Approach for Histology-Based Nuclei Segmentation and Tumor Microenvironment Characterization"
collection: publications
permalink: /publication/2022-12-08-hd-yolo
date: 2022-12-08
venue: 'Modern Pathology'
citation: 'Rong et al. (2023). &quot;A Deep Learning Approach for Histology-Based Nuclei Segmentation and Tumor Microenvironment Characterization.&quot; <i>Modern Pathology</i>. 1(3).'
---
Recommended citation: Rong et al. (2023). "A Deep Learning Approach for Histology-Based Nuclei Segmentation and Tumor Microenvironment Characterization." <i>Modern Pathology</i>. 1(3).